In [69]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

In [70]:
import os
from keras.models import Sequential # Initialise our neural network model as a sequential network
from keras.layers import Conv2D # Convolution operation
from keras.layers import MaxPooling2D # Maxpooling function
from keras.layers import Flatten # Converting 2D arrays into a 1D linear vector.
from keras.layers import Dense # Perform the full connection of the neural network
from keras.preprocessing.image import ImageDataGenerator
from IPython.display import display
from PIL import Image
import cv2
import numpy as np
from sklearn.metrics import accuracy_score
from skimage import io, transform

In [71]:
def cnn_classifier():
    cnn = Sequential()
    cnn.add(Conv2D(32, (3,3), input_shape = (24, 24, 3), padding='same', activation = 'relu'))
    cnn.add(MaxPooling2D(pool_size=(2, 2), padding='same'))
    cnn.add(Conv2D(64, (3,3), padding='same', activation = 'relu'))
    cnn.add(MaxPooling2D(pool_size=(2, 2), padding='same'))
    cnn.add(Flatten())
    cnn.add(Dense(500, activation = 'relu'))
    cnn.add(Dense(2, activation = 'softmax'))
    cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    print(cnn.summary())
    return cnn

def reshaped_image(image):
    return transform.resize(image,(24, 24, 3)) # (cols (width), rows (height)) and don't use np.resize()
   
def load_images_from_folder(dbFolder):
    train_images = []
    train_labels = []
    l = [0,0] # [False,True]
    
    Images = os.listdir(dbFolder + "/0/") 
    
    for image in Images:
        path = os.path.join(dbFolder + "/0/", image)
        img = cv2.imread(path)
        train_images.append(reshaped_image(img))
        l = [1,0] 
        train_labels.append(l)

    Images = os.listdir(dbFolder + "/1/")    
    for image in Images:
        ath = os.path.join(dbFolder + "/1/", image)
        img = cv2.imread(path)
        train_images.append(reshaped_image(img))
        l = [0,1]
        train_labels.append(l)
        
    return np.array(train_images), np.array(train_labels)
        
def train_test_split(train_data, train_labels, fraction):
    index = int(len(train_data)*fraction)
    return train_data[:index], train_labels[:index], train_data[index:], train_labels[index:]

train_data, train_labels = load_images_from_folder("datasets")
fraction = 0.8
train_data, train_labels, test_data, test_labels = train_test_split(train_data, train_labels, fraction)
print ("Train data size: ", len(train_data))
print ("Test data size: ", len(test_data))

cnn = cnn_classifier()

print ("Train data shape: ", train_data.shape)
print ("Test data shape: ", test_data.shape)

idx = np.random.permutation(train_data.shape[0])
cnn.fit(train_data[idx], train_labels[idx], batch_size = 64, epochs = 3)
predicted_test_labels = np.argmax(cnn.predict(test_data), axis=1)
test_labels = np.argmax(test_labels, axis=1)

print ("Actual test labels:", test_labels)
print ("Predicted test labels:", predicted_test_labels)

print ("Accuracy score:", accuracy_score(test_labels, predicted_test_labels))

Train data size:  5065
Test data size:  1267
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_17 (Conv2D)           (None, 24, 24, 32)        896       
_________________________________________________________________
max_pooling2d_17 (MaxPooling (None, 12, 12, 32)        0         
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 12, 12, 64)        18496     
_________________________________________________________________
max_pooling2d_18 (MaxPooling (None, 6, 6, 64)          0         
_________________________________________________________________
flatten_9 (Flatten)          (None, 2304)              0         
_________________________________________________________________
dense_17 (Dense)             (None, 500)               1152500   
_________________________________________________________________
dense_18 (Dense)             (N

In [72]:
# serialize model to JSON
model_json = cnn.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
cnn.save_weights("model.h5")
print("Saved model to disk")

Saved model to disk


In [73]:
from keras.models import model_from_json

# load json and create model
json_file = open('./model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("./model.h5",by_name=True)
print("Loaded model from disk")
 

Loaded model from disk


In [87]:
# evaluate loaded model on test data
loaded_model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

test_image = []
#img = cv2.imread("datasets/1/csi_1600x1600_20180614_083740.png-1.jpg")
#img = cv2.imread("datasets/1/csi_1600x1600_20180608_084252.png-3.jpg")
#img = cv2.imread("datasets/0/1529122929.7221885.jpg")
img = cv2.imread("datasets/0/1529122929.7362773.jpg")
test_image.append(reshaped_image(img))
test_image = np.array(test_image)
print(test_image.shape)
result = loaded_model.predict(test_image)
print(result)
print( np.argmax(result, axis=1) )

(1, 24, 24, 3)
[[3.066376e-27 1.000000e+00]]
[1]


In [85]:
print(result)
print( np.argmax(result, axis=1) )

[[4.5869545e-15 1.0000000e+00]]
[1]


In [83]:
print(np.argmax(np.array([4.2, 3.4]), axis=0))

0
